In [ ]:
import pathlib
import tifffile
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy import stats
from scipy import io
from matplotlib import pyplot as plt
import hd_functions as hd

In [ ]:
# Set experiment details.
superdir = 'C:/Users/ah-pc/Documents/tmp-2p'
#superdir = '/Users/andrewhill/Fisher Lab Dropbox/Andrew Hill/2p-data-processed'
expt = '251018-SS82773-P2X2-EPGEL-jGC7b'
fly = 'fly1'
session = f'{fly}-003'
rig = 'G' # 'S' for Smaug or 'G' for Galadriel

# Make paths.
exptdir = pathlib.PurePath(f'{superdir}/{expt}/{fly}/{session}-pv')

In [ ]:
rawF = np.load(exptdir.joinpath(f'{session}-F-array.npy'))

num_rois = len(rawF[0, :])
num_cycles = len(rawF)

rawF.shape

In [ ]:
# Convert raw fluorescence to ∆F/Fo with Fo equal to bottom 5th percentile of F values per glom.

Fo_cutoff = 0.05 # Bottom fifth percentile?
Fo = np.quantile(rawF, Fo_cutoff, axis = 0)

dFF = np.zeros([num_cycles, num_rois])
for r in range(num_rois):
    dFF[:, r] = (rawF[:, r] - Fo[r])/(Fo[r])

fig, axs = plt.subplots(num_rois, 1, figsize = (10,8))
for r in range(num_rois):
    ax = axs[r]
    ax.plot(dFF[:, r])
plt.show()

In [ ]:
# Normalize ∆F/Fo to top 95th percentile of F values per glom.

normF_cutoff = 0.95
normF_factor = np.quantile(dFF, normF_cutoff, axis = 0)

normF = np.zeros([num_cycles, num_rois])
for r in range(num_rois):
    normF[:, r] = dFF[:, r]/normF_factor[r]

fig, axs = plt.subplots(num_rois, 1, figsize = (10,8))
for r in range(num_rois):
    ax = axs[r]
    ax.plot(normF[:, r])
plt.show()

In [ ]:
# Median filter the normalized F. 

for r in range(num_rois):
    normF[:, r] = signal.medfilt(normF[:, r], kernel_size = 3)

fig, axs = plt.subplots(num_rois, 1, figsize = (10,8))
for r in range(num_rois):
    ax = axs[r]
    ax.plot(normF[:, r])
plt.show()

In [ ]:
normF_dataframe = pd.DataFrame(normF, columns = ['L8', 'L7', 'L6', 'L5', 'L4', 'L3', 'L2' ,'L1',
                                                 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8'])

normF_corr_matrix = normF_dataframe.corr()

plt.figure(figsize = (5,5))
sns.heatmap(normF_corr_matrix)
plt.xlabel('ROI ID', fontsize = 10)
plt.ylabel('ROI ID', fontsize = 10)
plt.title('Normalized ∆F/Fo Correlation Matrix', fontsize = 15)
plt.show()

In [ ]:
roi_mask = tifffile.imread(exptdir.joinpath(f'{session}-ROI-mask.tif'))
roi_mask = np.squeeze(roi_mask)

num_merged_rois = int(num_rois/2)
normF_merged_bridges = hd.merge_gloms(normF, roi_mask, num_merged_rois)

fig, axs = plt.subplots(num_merged_rois, 1, figsize = (10,6))
for r in range(num_merged_rois):
    ax = axs[r]
    ax.plot(normF_merged_bridges[:, r])
plt.show()

In [ ]:
# PVA

PVA_rad, PVA_str = hd.PVA_calc(normF_merged_bridges)

plt.figure(figsize = (40,8))
plt.scatter(np.arange(len(PVA_rad)), PVA_rad, color = 'black')
plt.show()

In [ ]:
normF_for_plot = normF_merged_bridges.transpose()
normF_for_plot_shifted = np.zeros((np.size(normF_for_plot, 0), np.size(normF_for_plot, 1)))

for r in range(len(normF_for_plot)):
    normF_for_plot_shifted[r, :] = normF_for_plot[(r+4)%8, :]

plt.figure(figsize = (40,8))
plt.imshow(normF_for_plot_shifted, aspect = 'auto', cmap = 'Blues')
plt.xlabel('Frame', fontsize = 40)
plt.ylabel('Bump position', fontsize = 40)
plt.xticks(ticks = plt.xticks()[0][0:], labels = np.array(plt.xticks()[0][0:], dtype = np.int64), fontsize = 30)
plt.yticks(ticks = plt.yticks()[0][0:], labels = np.array(plt.yticks()[0][0:]+1, dtype = np.int64), fontsize = 30)
plt.xlim(0, num_cycles)
plt.ylim(0,7)

plt.scatter(np.arange(len(PVA_rad)), ((PVA_rad+np.pi)/(2*np.pi))*7, color = 'black')

plt.show()

In [ ]:
rawV = pd.read_csv(exptdir.joinpath(f'{session}_Cycle00001_VoltageRecording_001.csv'))
rawV.shape

In [ ]:
if rig == 'S':
    raw_heading = rawV[' Heading']
if rig == 'G':
    raw_heading = rawV[' Input 4']
    
filt_heading = hd.low_pass_filter(raw_heading, 25, 50000)

In [ ]:
heading = hd.downsample_to_vols(filt_heading, num_cycles)

if rig == 'S':
    heading = ((heading/10)*-2*np.pi)+np.pi
if rig == 'G':
    heading = ((heading/10)* 2*np.pi)-np.pi

fig, axs = plt.subplots(2, 1, figsize = (12,4))
axs[0].plot(PVA_rad); axs[1].set_ylim(-np.pi, np.pi)
axs[1].plot(heading); axs[0].set_ylim(-np.pi, np.pi)
plt.show()

In [ ]:
heading2PVA_offset = (heading - PVA_rad)
for t in range(len(heading2PVA_offset)):
    if heading2PVA_offset[t] <= -np.pi:
        heading2PVA_offset[t] += (2*np.pi)
    if heading2PVA_offset[t] >= np.pi:
        heading2PVA_offset[t] -= (2*np.pi)

plt.figure(figsize = (12,2))
plt.plot(heading2PVA_offset)
plt.ylim(-np.pi, np.pi)
plt.show()

In [ ]:
pi_bins = [np.pi*(-8/8),
           np.pi*(-7/8),
           np.pi*(-6/8),
           np.pi*(-5/8),
           np.pi*(-4/8),
           np.pi*(-3/8),
           np.pi*(-2/8),
           np.pi*(-1/8),
           np.pi*(0),
           np.pi*(1/8),
           np.pi*(2/8),
           np.pi*(3/8),
           np.pi*(4/8),
           np.pi*(5/8),
           np.pi*(6/8),
           np.pi*(7/8),
           np.pi*(8/8)]
plt.hist(heading2PVA_offset, bins = pi_bins)
plt.show()

In [ ]:
# Circular variance:
offset_var = stats.circvar(heading2PVA_offset)
offset_var

In [ ]:
np.save(exptdir.joinpath(f'{session}-normF-8ROI-array.npy'), normF_merged_bridges)
np.save(exptdir.joinpath(f'{session}-PVA-rad-array.npy'), PVA_rad)
np.save(exptdir.joinpath(f'{session}-PVA-str-array.npy'), PVA_str)
np.save(exptdir.joinpath(f'{session}-heading-array.npy'), heading)

io.savemat(exptdir.joinpath(f'{session}-normF-8ROI-array.mat'), {'normF_8ROI': normF_merged_bridges})
io.savemat(exptdir.joinpath(f'{session}-PVA-rad-array.mat'), {'PVA_rad': PVA_rad})
io.savemat(exptdir.joinpath(f'{session}-PVA-str-array.mat'), {'PVA_str': PVA_str})
io.savemat(exptdir.joinpath(f'{session}-heading-array.mat'), {'heading': heading})